<h1>Data Scraping a Twice lyrics dataset</h1>
<br/>
This is my own project for practicing data scraping. This notebook builds a dataset with all Twice song lyrics romanized, and in Korean/Japanese and English. The complete dataset is here on Kaggle. Lyric data is from azlyrics.com.

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import time
import numpy as np

In [2]:
# Getting all album titles, publication years and urls of each song.

all_songs = 'https://www.azlyrics.com/t/twice.html'

albums = requests.get(all_songs)
soup = BeautifulSoup(albums.content.decode('utf-8'), 'html.parser')
albums.close()

albumList = soup.findAll('div', attrs={'id':'listAlbum'})

# fixing some characters
newlist = str(albumList).replace('&amp;', '&').replace('˂','<').split('div class="album"')


Under I'm listing all albums where one list element is an album ```[name, year, [list of songs]```.

In [3]:
all_albums = []
one_album = []

for i in newlist:
    
    if '<b>\"' in i:
        tup1 = re.findall(r'<b>"(.*?)"</b> [(](.*?)[)]<div>', i)
        one_album.append(tup1[0][0]) # name
        one_album.append(tup1[0][1]) # year
    
    if 'href="/lyrics' in i:
        tup2 = re.findall(r'href="/(.*?)" tar', i)
        one_album.append(tup2) # url to song list

    if one_album != []:
        all_albums.append(one_album)
    one_album = []
        

Some published songs don't belong to any album. Those songs are all in the same list, which is the last element in ```all_albums```, and that list item does not have album name nor year.

In [4]:
number_of_songs = 0

for album in all_albums:
    for song in album[len(album)-1]:
        number_of_songs += 1
print('Number of songs:', number_of_songs)

Number of songs: 228


In [5]:
# The final dataframe
df = pd.DataFrame(columns=['Id','Album','Year','Song','url','Romanized_lyrics','Korean_lyrics','Japanese_lyrics','English_lyrics'])

In [6]:
album_list_index = 0
df_id = 0
homepage = 'https://www.azlyrics.com/'

for a in range(0,len(all_albums)):
    
    if a == (len(all_albums)-1): 
    # This statement is needed for the last item in all_albums, which is just the song list
        album = all_albums[a]
        album_title = np.nan  # no album name
        year = np.nan         # no album year
        album_list_index = 0
        
    else: # This is the normal case
        album = all_albums[a]
        album_title = album[0]
        year = album[1]
        album_list_index = 2

    for url in album[album_list_index]: # For song in an album
        
        page = requests.get(homepage + url)
        soup = BeautifulSoup(page.content.decode('utf-8'), "html.parser")
        page.close()
        
        # lyrics
        lyrics = soup.findAll('div', attrs={'class':'col-xs-12 col-lg-8 text-center'})
        str_lyrics = str(lyrics)
        
        # song title 
        title = soup.findAll('h1')
        title = str(title).split('\"')[1]
        og_title = title
        title = title.replace('&amp;', '&').replace('˂','<')
        
        onlyenglish = np.nan
        romanized = np.nan
        korean = np.nan
        japanese= np.nan
        english= np.nan
        
        # This part of the code seems quite clumsy, but the format on the web page sometimes changes depending on
        # the song or available lyrics, and this is the best I came up with so far without having to
        # detect languages. Different lyrics are processed in the order where they usually appear. Variable
        # 'onlyenglish' is used for English songs, because the html format is slightly different for those.
        
        if '[Romanized' in str_lyrics:
            rom_lyr = str_lyrics
            rom_l = rom_lyr.split('[Romanized')
            rom_lyr = rom_l[1] 
            rom_lyr = rom_lyr.split('</div>')[0]
            if 'Korean' or 'English' or 'Japanese' in rom_lyr:
                rom_lyr = rom_lyr.split('[')[0]
            final_rom_lyr = re.sub('<.*>', '', rom_lyr).strip()
            final_rom_lyr = final_rom_lyr.strip(':').strip(']').strip()
            romanized = final_rom_lyr

        if '[Korean' in str_lyrics:
            kor_lyr = str_lyrics
            kor_l = kor_lyr.split('[Korean')
            kor_lyr = kor_l[1] 
            kor_lyr = kor_lyr.split('</div>')[0]
            if '[English' in kor_lyr:
                kor_lyr = kor_lyr.split('[E')[0]
            final_kor_lyr = re.sub('<.*>', '', kor_lyr).strip()
            final_kor_lyr = final_kor_lyr.strip(':').strip(']').strip()
            korean = final_kor_lyr

        if '[Japanese' in str_lyrics:
            jap_lyr = str_lyrics
            jap_l = jap_lyr.split('[Japanese')
            jap_lyr = jap_l[1] 
            jap_lyr = jap_lyr.split('</div>')[0]
            if '[English' in jap_lyr:
                jap_lyr = jap_lyr.split('[E')[0]
            final_jap_lyr = re.sub('<.*>', '', jap_lyr).strip()
            final_jap_lyr = final_jap_lyr.strip(':').strip(']').strip()
            japanese = final_jap_lyr

        if '[English' in str_lyrics:
            eng_lyr = str_lyrics
            eng_l = eng_lyr.split('[English')
            eng_lyr = eng_l[len(eng_l)-1] 
            eng_lyr = eng_lyr.split(']')[1] 
            eng_lyr = eng_lyr.split('</div>')[0]
            final_eng_lyr = re.sub('<.*>', '', eng_lyr).strip()
            final_eng_lyr = final_eng_lyr.strip(':').strip(']').strip()
            english = final_eng_lyr
    
        if english == np.nan and ('[Romanized' not in str_lyrics):
            lyr = str_lyrics
            l = lyr.split('<b>"' + og_title + '"</b><br/>')
            lyr = l[1]
            lyr = lyr.split('</div>')[0]
            final_lyr = re.sub('<.*>', '', lyr).strip()
            final_lyr = final_lyr.strip(':').strip(']').strip()
            onlyenglish = final_lyr
        
            data_item = [df_id, album_title, year, title, homepage + url, romanized, korean, japanese, onlyenglish]
        else:
            data_item = [df_id, album_title, year, title, homepage + url, romanized, korean, japanese, english]
            
        df.loc[len(df.index)] = data_item
        df_id += 1
        
        time.sleep(10)
        print(album_title,'-',title)
        print('Songs left:', number_of_songs - len(df))


The Story Begins - Like Ooh Ahh (OOH-AHH하게; Ooh-Ahh Hage)
Songs left: 227
The Story Begins - Do It Again (다시 해줘; Dasi Haejwo)
Songs left: 226
The Story Begins - Going Crazy (미쳤나봐; Michyeonnabwa)
Songs left: 225
The Story Begins - Truth
Songs left: 224
The Story Begins - Candy Boy
Songs left: 223
The Story Begins - Like A Fool
Songs left: 222
Page Two - Cheer Up
Songs left: 221
Page Two - Precious Love (소중한 사랑; Sojunghan Sarang)
Songs left: 220
Page Two - Touchdown
Songs left: 219
Page Two - Tuk Tok (툭하면 톡; Tukhamyeon Tok)
Songs left: 218
Page Two - Woohoo
Songs left: 217
Page Two - My Headphones On (Headphone 써; Headphone Sseo)
Songs left: 216
Page Two - I'm Gonna Be A Star
Songs left: 215
Twicecoaster: Lane 1 - TT
Songs left: 214
Twicecoaster: Lane 1 - 1 To 10
Songs left: 213
Twicecoaster: Lane 1 - Ponytail
Songs left: 212
Twicecoaster: Lane 1 - Jelly Jelly
Songs left: 211
Twicecoaster: Lane 1 - Pit-A-Pat
Songs left: 210
Twicecoaster: Lane 1 - Next Page
Songs left: 209
Twicecoaster: L

Formula Of Love: O+T=<3 - The Feels
Songs left: 46
Formula Of Love: O+T=<3 - SCIENTIST (R3HAB Remix)
Songs left: 45
Doughnut - Doughnut
Songs left: 44
Doughnut - Wonderful Day
Songs left: 43
#TWICE4 - I CAN'T STOP ME (Japanese Version)
Songs left: 42
#TWICE4 - CRY FOR ME (Japanese Version)
Songs left: 41
#TWICE4 - Alcohol-Free (Japanese Version)
Songs left: 40
#TWICE4 - SCIENTIST (Japanese Version)
Songs left: 39
Celebrate - Celebrate
Songs left: 38
Celebrate - Voices Of Delight
Songs left: 37
Celebrate - TICK TOCK
Songs left: 36
Celebrate - Flow Like Waves
Songs left: 35
Celebrate - That's All I'm Saying
Songs left: 34
Celebrate - Bitter Sweet
Songs left: 33
Celebrate - Sandcastle
Songs left: 32
Celebrate - Just Be Yourself
Songs left: 31
Celebrate - Doughnut
Songs left: 30
Between 1&2 - Talk That Talk
Songs left: 29
Between 1&2 - Queen Of Hearts
Songs left: 28
Between 1&2 - Basics
Songs left: 27
Between 1&2 - Trouble
Songs left: 26
Between 1&2 - Brave
Songs left: 25
Between 1&2 - Gon

In [7]:
df.to_csv('twice_lyrics_dataset1.csv', index=False)

Finished!\
Let's look at some example.

In [9]:
# English lyrics of Cheer up
print(df['English_lyrics'].iloc[6])

Every day it rings, the bell bell bell
Now care for me
I don't wanna waste my battery
It keeps coming, it keeps coming
Like my phone's gonna explode
I don't know, I don't know, he says he can't even breathe
He says he's struggling because of me
He says his heart is dropping, why?
He says that I'm so pretty
I'm not showing off or anything

Sorry I couldn't pick up earlier
I was meeting my friends, shy shy shy
I can't really meet you right now, sorry
I'll call you a little later

Don't beg, then you won't last long
I'll let you call me baby
It's still a little too early, too early for my heart
To feel the same – But I'll show you more

CHEER UP BABY CHEER UP BABY
Cheer up a little more
A girl can't give her heart too easily
That's how you'll get to like me even more
I'll act calm, as if it's nothing
So you won't know that I like you
Just get it together
And then baby CHEER UP

I can hear the anxiety in your voice
I can see the sweat forming on your phone
It's not attractive to answer rig

In [11]:
# Korean lyrics for TT
print(df.iloc[13,6])

이러지도 못하는데
저러지도 못하네
그저 바라보며 ba-ba-ba-baby
매일 상상만 해 이름과 함께
쓱 말을 놨네 baby
아직 우린 모르는 사인데
아무거나 걸쳐도 아름다워
거울 속 단 둘이서 하는
Fashion show show
이번엔 정말 꼭꼭 내가 먼저 talk talk
다짐 뿐인걸 매번 다짐 뿐인걸

나나나나나나나
콧노래가 나오다가 나도 몰래
눈물 날 것 같애
아닌 것 같애 내가 아닌 것 같애
I love you so much

이미 난 다 컸다고 생각하는데
어쩌면 내 맘인데 왜
내 맘대로 할 수 없는 건데
밀어내려고 하면 할수록
자꾸 끌려 왜 자꾸 자꾸 끌려 baby

I'm like TT, Just like TT
이런 내 맘 모르고 너무해 너무해
I'm like TT, Just like TT
Tell me that you'll be my baby

어처구니 없다고 해
얼굴 값을 못한대
전혀 위로 안돼 ba-ba-ba-baby
미칠 것 같애 이 와중에 왜
배는 또 고픈 건데
하루 종일 먹기만 하는데

맴매매매 아무 죄도 없는
인형만 때찌
종일 앉아있다가 엎드렸다
시간이 획획획
피부는 왜 이렇게 또 칙칙
자꾸 틱틱 거리고 만 싶지
엄만 귀찮게 계속 왜왜왜왜왜

나나나나나나나
콧노래가 나오다가 나도 몰래
짜증날 것 같애 화날 것 같애
이런 애가 아닌데
I love you so much

이미 난 다 컸다고 생각하는데
어쩌면 내 맘인데 왜
내 맘대로 할 수 없는 건데
밀어내려고 하면 할수록
자꾸 끌려 왜 자꾸 자꾸 끌려 baby

I'm like TT, Just like TT
이런 내 맘 모르고 너무해 너무해
I'm like TT, Just like TT
Tell me that you'll be my baby

혹시 이런 나를 알까요
이대로 사라져 버리면 안돼요
이번엔 정말 꼭꼭 내가 먼저 talk talk
다짐 뿐인걸 매번 다짐 뿐인걸

이미 난 다 컸다고 생각하는데
어쩌면 내 맘인데 왜
내 맘대로 할 수 없는 건데
밀어내려고 하면 할수록
자꾸 끌려 왜